In [2]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.0/163.0 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.8/479.8 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 18.3 MB/s eta 0

In [ ]:
from langchain.llms import GooglePalm

api_key = 'your_api_key'

llm = GooglePalm(google_api_key=api_key, temperature=0.2)

In [ ]:
# print(test)

In [3]:
from langchain.utilities import SQLDatabase

#enter your details in these variables
db_user = '*************'
db_password = '*************'
db_host = '*************'
db_name = '*************'


db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER(11) NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER(11) NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id)
)DEFAULT CHARSET=latin1 ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER(11) NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER(11), 
	stock_quantity INTEGER(11) NOT NULL, 
	PRIMARY KEY (t_shirt_id)
)DEFAULT CHARSET=latin1 ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Adidas	White	S	15	72
2	Van Huesen	Red	L	24	50
3	Van Huesen	Blue	M	41	10
*/


In [4]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain.run("How many t-shirts do we have left for nike in extra small size and black color?")



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and black color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'Black' AND size = 'XS'
SQLResult: [(97,)]
Answer:97
> Finished chain.


In [5]:
qns1

'97'

In [6]:
qns2 = db_chain.run("How much is the price of all small size t-shirts in the inventory?")



> Entering new SQLDatabaseChain chain...
How much is the price of all small size t-shirts in the inventory?
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('381'),)]
Answer:381
> Finished chain.


In [7]:
qns3 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size='S'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size='S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size='S'
SQLResult: [(Decimal('17349'),)]
Answer:17349
> Finished chain.


In [8]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('20580.800000'),)]
Answer:20580.8
> Finished chain.


In [9]:
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('22241'),)]
Answer:22241
> Finished chain.


In [10]:
qns5 = db_chain.run("How many white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts we have available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE color = 'White' AND brand = 'Levi'
SQLResult: [(58,), (56,), (96,)]
Answer:58
> Finished chain.


## Few shot learning

In [11]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1},

    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},

    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
     (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
     group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
     """,
     'SQLResult': "Result of the SQL query",
     'Answer': qns3} ,

     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns4},

    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5
     }
]

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [13]:
e = embeddings.embed_query("How many white color Levi's shirt I have?")

In [14]:
len(e)

384

In [15]:
to_vectorize = [' '.join(example.values()) for example in few_shots]

In [40]:
to_vectorize[3]

"If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount? SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi' Result of the SQL query 22241"

In [16]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_texts(to_vectorize, embedding=embeddings, metadatas=few_shots)

In [17]:
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(
    vectorstore= vectorstore,
    k=2
)

example_selector.select_examples({"Question": "How many Addidas T shirts I have left in my store?"})

[{'Answer': '97',
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '58',
  'Question': "How many white color Levi's shirt I have?",
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
  'SQLResult': 'Result of the SQL query'}]

In [18]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

In [19]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [20]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [21]:
# print(table_info)

In [22]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult", "Answer",],
    template = "\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}"
)

In [23]:
from langchain.prompts import FewShotPromptTemplate

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=_mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"]
)

In [24]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [25]:
new_chain("How many white color Levi's tshirt I have?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's tshirt I have?
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('210'),)]
Answer:210
> Finished chain.


{'query': "How many white color Levi's tshirt I have?", 'result': '210'}

In [26]:
new_chain("If I have to sell all the items present in the inventory, after applying the respective discounts, how much revenue will be generated?")



> Entering new SQLDatabaseChain chain...
If I have to sell all the items present in the inventory, after applying the respective discounts, how much revenue will be generated?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from 
    (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('92237.100000'),)]
Answer:92237.1
> Finished chain.


{'query': 'If I have to sell all the items present in the inventory, after applying the respective discounts, how much revenue will be generated?',
 'result': '92237.1'}

In [40]:
!pip install -q streamlit

In [27]:
import streamlit as st

In [33]:
%%writefile main.py

import streamlit as st

st.title("Musa stores: Database Q&A")

question = st.text_input("Question: ")
if question:
  chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)
  response = chain.run(question)
  st.header("Answer")
  st.write(response)

Overwriting main.py


In [ ]:
import streamlit as st

st.title("Musa stores: Database Q&A")

question = st.text_input("Question: ")
if question:
  chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)
  response = chain.run(question)
  st.header("Answer")
  st.write(response)

In [34]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.527s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [35]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.27.219.84


In [36]:
!streamlit run main.py &>/content/logs.txt &

In [37]:
!npx localtunnel --port 8501

npx: installed 22 in 2.186s
your url is: https://full-garlics-make.loca.lt
^C


In [38]:
!pwd

/content
